In [6]:
from openai import OpenAI
import json
import os

In [7]:
API_KEY = 'YOUR_API_KEY'
MODEL = 'gpt-4o-mini'
ASSISTANT_ID = 'YOUR_ASSISTANT_ID'

In [8]:
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [9]:
def find_files_with_extensions(directory, extensions):

    matched_files = []    
    for root, _, files in os.walk(directory):
        for file in files:
            if any(file.endswith(ext) for ext in extensions):
                matched_files.append(os.path.join(root, file))
                
    return matched_files

In [11]:
client = OpenAI(api_key=API_KEY)

### Step 1: Create a new Assistant with File Search Enabled

In [ ]:
assistant = client.beta.assistants.create(
  name="AI Bot Creator",
  instructions="You are an expert financial analyst for Companies 10k documents. You will use file_search tool to retrieve latest data from companies finantial reports 10k and use you knowledge base to answer questions about these financial statements.In case if you need to do calculations on numbers from these documents you will use code_interpreter tool",
  model=MODEL,
  tools=[{"type": "file_search"}, {"type": "code_interpreter"}],
)

In [ ]:
show_json(assistant)

### Create Vector Store

In [13]:
vector_store = client.beta.vector_stores.create(name="AI bot Sc2")

### Load Files into the vector store

In [14]:
file_path1 = find_files_with_extensions('python-sc2/sc2',['.py','.md'])
file_path2 = find_files_with_extensions('python-sc2/examples',['.py','.md'])
file_paths = file_path1 + file_path2 + ['python-sc2/README.md']

85

In [ ]:
len(file_paths)

In [15]:
file_paths

['python-sc2/sc2/expiring_dict.py',
 'python-sc2/sc2/game_state.py',
 'python-sc2/sc2/maps.py',
 'python-sc2/sc2/sc2process.py',
 'python-sc2/sc2/bot_ai.py',
 'python-sc2/sc2/controller.py',
 'python-sc2/sc2/units.py',
 'python-sc2/sc2/paths.py',
 'python-sc2/sc2/renderer.py',
 'python-sc2/sc2/wsl.py',
 'python-sc2/sc2/proxy.py',
 'python-sc2/sc2/protocol.py',
 'python-sc2/sc2/client.py',
 'python-sc2/sc2/observer_ai.py',
 'python-sc2/sc2/constants.py',
 'python-sc2/sc2/power_source.py',
 'python-sc2/sc2/cache.py',
 'python-sc2/sc2/__init__.py',
 'python-sc2/sc2/portconfig.py',
 'python-sc2/sc2/generate_ids.py',
 'python-sc2/sc2/action.py',
 'python-sc2/sc2/unit_command.py',
 'python-sc2/sc2/bot_ai_internal.py',
 'python-sc2/sc2/game_data.py',
 'python-sc2/sc2/player.py',
 'python-sc2/sc2/unit.py',
 'python-sc2/sc2/position.py',
 'python-sc2/sc2/game_info.py',
 'python-sc2/sc2/versions.py',
 'python-sc2/sc2/pixel_map.py',
 'python-sc2/sc2/main.py',
 'python-sc2/sc2/data.py',
 'python-s

In [16]:
file_streams = [open(path, "rb") for path in file_paths]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=85, failed=0, in_progress=0, total=85)


### Connect Vector Store with Assistant

In [17]:
assistant = client.beta.assistants.update(
  assistant_id=ASSISTANT_ID,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)